# Multi-Armed-Bandit Problem

In [3]:
#Library
import random
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## agent(環境)
今回の場合はスロットを用意する  
報酬周りがよくわからない(´･ω･｀)  現在はベルヌーイ分布の結果を報酬としている.. 

### ベルヌーイ分布

$$
    f(k;p) = \begin{cases}
        p  & k=1\\
        1-p & k=0
    \end{cases}
$$

In [407]:
## 当たるor当たらない
class BernoulliArm():
    def __init__(self,p):
        self.p = p
    
    def draw(self):
        if random.random() > self.p:
            return 0.0
        else:
            return 1.0

### ガウシアン分布

$$ \frac{1}{\sqrt{2 \pi \sigma^2}} exp(- \frac{(x - \mu)^2}{2\sigma^2}) $$ 

In [408]:
## 正規分布
class NormalArm():
    def __init__(self,mu,sigma):
        self.mu = mu
        self.sigma = sigma
    
    def draw(self):
        return random.gauss(self.mu,self.sigma)

In [409]:
## sample(標準正規分布)
arm_test = NormalArm(0,1)
arm_test.draw()

-1.438368849514714

### greedy algorithm
まだn回選んだことがない腕がある場合, その腕を選出  
それ以外の場合, 全ての腕に対して, これまでの報酬の平均を計算する  
  
$$ u_i = \frac{これまで腕iから得られた報酬の和}{これまで腕iをプレイした回数} $$  

u_i が最大の腕を選ぶ

In [410]:
#リスト内の最大値または最小値のインデックスを返す関数
def ind(x,option="max"):
    if option=="min":
        m = min(x)
    else:
        m = max(x)
    return x.index(m)

In [533]:
class Greedy():
    def __init__(self,counts,values):
        self.counts = counts
        self.values = values
        
    def initialize(self,n_arms):
        self.counts = [0 for col in range(n_arms)]
        self.values = [0.0 for col in range(n_arms)]
    
    def select_arm(self):
        count_min = min(self.counts)
        #n回に達していない場合は達していないマシンの中で一番回数の低いマシンを選出
        if count_min < n:
            return self.counts.index(count_min)
        #全て達している場合には報酬の最大になるマシンを選出
        else:
            return ind(self.values)
            
    def update(self,choose_arm,reward):
        self.counts[choose_arm] = self.counts[choose_arm] +1 
        value = self.values[choose_arm] #選んだマシンの報酬
        #new_value = (value + reward)/ self.counts[choose_arm]
        new_value = ((n-1)/float(n))*value + (1/float(n))*reward
        self.values[choose_arm] = new_value

### ε - greedy algorithm
まだ選んだことがない腕がある場合、その腕から一つ選ぶ  
確率εで、すべての腕からランダムに一つ選ぶ  
確率１−εで、これまでの報酬の平均u_i が最大の腕を選ぶ  

In [534]:
class epsilon_Greedy():
    def __init__(self,epsilon,counts,values):
        self.epsilon = epsilon
        self.counts = counts
        self.values = values
        
    def initialize(self,n_arms):
        self.counts = [0 for col in range(n_arms)]
        self.values = [0.0 for col in range(n_arms)]
    
    def select_arm(self):
        if random.random() > self.epsilon:
            return ind(self.values)
        #全て達している場合には報酬の最大になるマシンを選出
        else:
            return random.randrange(len(self.values))
            
    def update(self,choose_arm,reward):
        self.counts[choose_arm] = self.counts[choose_arm] +1 
        value = self.values[choose_arm] #選んだマシンの既存報酬
        #new_value = (value + reward)/ self.counts[choose_arm]
        new_value = ((n-1)/float(n))*value + (1/float(n))*reward
        self.values[choose_arm] = new_value

### demo
スロットマシンを3台, 各マシンを3回ずつ回す(初期設定)

In [535]:
armTest = BernoulliArm(0.6)
res=0
reaw=0
for i in range(100):
    tmpDraw=armTest.draw()
    reaw+=tmpDraw*300
    res+=tmpDraw
print(res/100,reaw/100)

0.6 180.0


In [549]:
armA = BernoulliArm(0.2)
armB = BernoulliArm(0.3)
armC = BernoulliArm(0.4)
armD = BernoulliArm(0.5)
#armC = BernoulliArm(0.2)
arms = [armA,armB,armC,armD]
n_arms = len(arms)


n = 100 #最初に探索する情報を取得

algo = Greedy([],[])
algo.initialize(n_arms)

'''
oxdict={0:"o",1:"x"}
print("施行,行動,結果")
for i in range(6):
    choose_arm = algo.select_arm()
    reward = arms[choose_arm].draw()
    algo.update(choose_arm,reward)
    print("{},{},{}".format(i,choose_arm,oxdict[reward]))

print("内訳:{} 価値{}".format(algo.counts,algo.values))

for j in range(10):
    choose_arm = algo.select_arm()
    reward = arms[choose_arm].draw()
    algo.update(choose_arm,reward)
    print("{},{},{}".format(i+j+1,choose_arm,oxdict[reward]))
    
print("内訳:{} 価値{}".format(algo.counts,algo.values))
'''
arm_A,arm_B,arm_C,arm_D = [],[],[],[]
for i in range(10000):
    choose_arm = algo.select_arm()
    reward = arms[choose_arm].draw()
    algo.update(choose_arm,reward)
    
print("内訳:{} 価値{}".format(algo.counts,algo.values))
np.array(algo.counts)/10000 * 100

内訳:[100, 100, 100, 9700] 価値[0.19863667250041142, 0.20264185047914524, 0.26472267933149707, 0.4861824209836211]


array([  1.,   1.,   1.,  97.])

In [550]:
armA = BernoulliArm(0.2)
armB = BernoulliArm(0.3)
armC = BernoulliArm(0.4)
armD = BernoulliArm(0.5)
#armC = BernoulliArm(0.2)
arms = [armA,armB,armC,armD]
n_arms = len(arms)


n = 100 #最初に探索する情報を取得

algo = epsilon_Greedy(0.1,[],[])
algo.initialize(n_arms)
for i in range(10000):
    choose_arm = algo.select_arm()
    reward = arms[choose_arm].draw()
    algo.update(choose_arm,reward)
    
print("内訳:{} 価値{}".format(algo.counts,algo.values))
np.array(algo.counts)/10000 * 100

内訳:[1896, 236, 3768, 4100] 価値[0.19904694558896596, 0.2869443250735782, 0.3973231087963187, 0.5167811277932451]


array([ 18.96,   2.36,  37.68,  41.  ])

## Upper Confidence Bound(UCB) Algorithm  
得られた評価値の信頼性を自動的に高めようとする. あまりよくわかっていない腕は積極的に使用する.

#### ~ Algotirhm
R: 振戻額の最大値と最小値の差  
まだ選出したことがない腕があれば, そのうちの一つを選ぶ  
各々の腕iから得られる報酬の期待値を計算する  

$$ u_i = \frac{これまで腕iから得られた報酬の和}{これまで腕iを選んだ回数} $$ 

各々の腕iから得られる報酬の信頼区間の半幅を計算する  

$$ U_i = R \sqrt{\frac{2 ln(これまでの総プレイ回数)}{これまで腕iをプレイした回数}} $$

そして
$$ u_i + U_i  が最大の腕 i を選出する $$

In [538]:
sum(algo.counts)

10000

In [539]:
import math

class UCB():
    def __init__(self,counts,values):
        self.counts = counts
        self.values = values
    
    def initialize(self,n_arms):
        self.counts = [0 for col in range(n_arms)]
        self.values = [0.0 for col in range(n_arms)]
    
    def select_arm(self):
        n_arms = len(self.counts)
        for arm in range(n_arms):
            if self.counts[arm] == 0:
                return arm
        ucb_values = [0.0 for arm in range(n_arms)]
        total_counts = sum(self.counts)
        
        for arm in range(n_arms):
            bonus = math.sqrt((2*math.log(total_counts))/ float(self.counts[arm]))
            ucb_values[arm] = self.values[arm] + bonus
        
        return ind(ucb_values)
    
    def update(self,chosen_arm,reward):
        self.counts[chosen_arm] = self.counts[chosen_arm] +1
        n = self.counts[chosen_arm]
        
        value = self.values[chosen_arm]
        new_value = ((n-1)/float(n))*value + (1/float(n))*reward
        self.values[chosen_arm] = new_value
        

In [540]:
armA = BernoulliArm(0.2)
armB = BernoulliArm(0.3)
armC = BernoulliArm(0.4)
armD = BernoulliArm(0.5)
#armC = BernoulliArm(0.2)
arms = [armA,armB,armC,armD]
n_arms = len(arms)


n = 100 #最初に探索する情報を取得

algo = UCB([],[])
algo.initialize(n_arms)
for i in range(10000):
    choose_arm = algo.select_arm()
    reward = arms[choose_arm].draw()
    algo.update(choose_arm,reward)
    
print("内訳:{} 価値{}".format(algo.counts,algo.values))
np.array(algo.counts)/10000 * 100

内訳:[181, 316, 1248, 8255] 価値[0.22651933701657467, 0.30379746835443056, 0.42387820512820484, 0.498606904906118]


array([  1.81,   3.16,  12.48,  82.55])